In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [19]:
seed = 42

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [20]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [21]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [22]:
def insert_element_randomly(my_list, element):
    
    if len(my_list) > 1:

        index = random.randint(0, 2)
        
    else:
        
        index = 0

    new_list = my_list[:(4 * index)] + element + my_list[(4 * index):]
    
    return new_list

In [23]:
N = 20 # vocab_size
M = 20 # number of random words

vocabs = ['word_' + str(i) for i in range(N)] + ['random_' + str(i) for i in range(M)]

vocabs_word = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i

In [24]:
def get_accuracy_prob(embed_dim):
    
    pairs = []
    for i in vocabs_word:
        for j in vocabs_word:
            for k in vocabs_word:
                if i != j and i != k and j != k:
                    pairs.append((i,j,k))

    indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

    pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
    pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

    sentences_train = []
    sentences_number_train = []
    sentences_test_a = []
    sentences_number_test_a = []
    sentences_test_b = []
    sentences_number_test_b = []

    x_masked_train = []
    y_masked_labels_train = []
    x_masked_test_a = []
    y_masked_labels_test_a = []
    x_masked_test_b = []
    y_masked_labels_test_b = []

    for _ in range(25000):

        random_words = random.sample(['random_' + str(i) for i in range(M)], 4)

        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, a, d, e, f, d]
        temp = insert_element_randomly(temp, random_words)

        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])

        random_words = random.sample(['random_' + str(i) for i in range(M)], 4)

        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, b, d, e, f, e]
        temp = insert_element_randomly(temp, random_words)

        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])



    for _ in range(25000):

        [(a,b,c), (d,e,f), (g,h,i)] = random.sample(pairs_test, 3)

        temp = [a, b, c, a, d, e, f, d, g, h, i, g]

        sentences_test_a.append(temp)
        sentences_number_test_a.append([vocab_map[i] for i in temp])
        x_masked_test_a.append([vocab_map[i] for i in temp])
        y_masked_labels_test_a.append([vocab_map[i] for i in temp][1:])

        [(a,b,c), (d,e,f), (g,h,i)] = random.sample(pairs_test, 3)

        temp = [a, b, c, b, d, e, f, e, g, h, i, h]

        sentences_test_b.append(temp)
        sentences_number_test_b.append([vocab_map[i] for i in temp])
        x_masked_test_b.append([vocab_map[i] for i in temp])
        y_masked_labels_test_b.append([vocab_map[i] for i in temp][1:])

    x_masked_train = np.array(x_masked_train)
    y_masked_labels_train = np.array(y_masked_labels_train)
    x_masked_test_a = np.array(x_masked_test_a)
    y_masked_labels_test_a = np.array(y_masked_labels_test_a)
    x_masked_test_b = np.array(x_masked_test_b)
    y_masked_labels_test_b = np.array(y_masked_labels_test_b)

    perm = np.random.permutation(len(x_masked_train))
    x_masked_train = x_masked_train[perm]
    y_masked_labels_train = y_masked_labels_train[perm]

    x_masked_train = x_masked_train[:,:-1]
    x_masked_test_a = x_masked_test_a[:,:-1]
    x_masked_test_b = x_masked_test_b[:,:-1]
    
    num_heads = 2
    num_blocks = 5

    batch_size = 1024

    input_layer = layers.Input(shape=(x_masked_train.shape[1],), dtype=tf.int32)  # Input layer

    embedding_layer = layers.Embedding(M + N, embed_dim, name="word_embedding")(input_layer)  # Embedding layer
    position_embeddings = PositionEmbedding(sequence_length=len(x_masked_train[0]))(embedding_layer)
    embedding_layer = embedding_layer + position_embeddings

    # Transformer blocks with causal masking for next token prediction
    x = embedding_layer
    for i in range(num_blocks):
        # Apply the causal mask to ensure that each position can only attend to known tokens
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim // num_heads
        )(x, x, x, use_causal_mask=True)

        x = layers.Add()([x, attention_output])  # Skip Connection
        x = layers.LayerNormalization(epsilon=1e-6)(x)

        ff_net = keras.models.Sequential([
            layers.Dense(2 * embed_dim, activation='relu'),
            layers.Dense(embed_dim),
        ])

        # Apply Feedforward network
        x = ff_net(x)

        # Add & Normalize
        x = layers.Add()([attention_output, x]) 
        x = layers.LayerNormalization(epsilon=1e-6)(x)

    # Output layer for providing predictions over the vocabulary
    predict_layer = layers.Dense(M + N, activation='softmax')(x)

    model = models.Model(inputs=input_layer, outputs=predict_layer)  # Model definition
    model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Compile the model

    # Reshape the target data to have an extra dimension
    y_masked_labels_train_reshaped = y_masked_labels_train.reshape(y_masked_labels_train.shape[0], 
                                                                   y_masked_labels_train.shape[1], 1)
    
    x_test_subset_a = x_masked_test_a[np.random.choice(x_masked_test_a.shape[0], size=500, replace=False)]
    x_test_subset_b = x_masked_test_b[np.random.choice(x_masked_test_b.shape[0], size=500, replace=False)]
    
    history = model.fit(x_masked_train, y_masked_labels_train_reshaped, epochs=100, batch_size=batch_size)
    
    acc_a = []
    prob_a = []

    for sentence_number in x_test_subset_a:
        temp = keras.backend.function(inputs = model.layers[0].input, outputs = model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_a.append(1 if temp.argmax() == sentence_number[-3] else 0)
        prob_a.append(temp[0][sentence_number[-3]])

    acc_b = []
    prob_b = []

    for sentence_number in x_test_subset_b:
        temp = keras.backend.function(inputs = model.layers[0].input, outputs = model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_b.append(1 if temp.argmax() == sentence_number[-2] else 0)
        prob_b.append(temp[0][sentence_number[-2]])
    
    return ((np.mean(acc_a), np.mean(prob_a)), (np.mean(acc_b), np.mean(prob_b)))

In [26]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(10)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

Epoch 1/100
49/49 [==============================] - 9s 55ms/step - loss: 3.7045 - accuracy: 0.0268
Epoch 2/100
49/49 [==============================] - 3s 54ms/step - loss: 3.6423 - accuracy: 0.0334
Epoch 3/100
49/49 [==============================] - 3s 53ms/step - loss: 3.6331 - accuracy: 0.0352
Epoch 4/100
49/49 [==============================] - 3s 54ms/step - loss: 3.5935 - accuracy: 0.0395
Epoch 5/100
49/49 [==============================] - 3s 53ms/step - loss: 3.2183 - accuracy: 0.0522
Epoch 6/100
49/49 [==============================] - 3s 54ms/step - loss: 3.0827 - accuracy: 0.0573
Epoch 7/100
49/49 [==============================] - 3s 53ms/step - loss: 3.0452 - accuracy: 0.0610
Epoch 8/100
49/49 [==============================] - 3s 53ms/step - loss: 3.0251 - accuracy: 0.0635
Epoch 9/100
49/49 [==============================] - 3s 53ms/step - loss: 3.0127 - accuracy: 0.0656
Epoch 10/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9978 - accuracy: 0.0687

49/49 [==============================] - 3s 54ms/step - loss: 2.8078 - accuracy: 0.0926
Epoch 63/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8084 - accuracy: 0.0921
Epoch 64/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8056 - accuracy: 0.0934
Epoch 65/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8045 - accuracy: 0.0935
Epoch 66/100
49/49 [==============================] - 3s 53ms/step - loss: 2.8024 - accuracy: 0.0936
Epoch 67/100
49/49 [==============================] - 3s 53ms/step - loss: 2.8043 - accuracy: 0.0935
Epoch 68/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8008 - accuracy: 0.0930
Epoch 69/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7992 - accuracy: 0.0943
Epoch 70/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8007 - accuracy: 0.0939
Epoch 71/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7978 - accuracy: 0.0939
Epo

49/49 [==============================] - 3s 53ms/step - loss: 2.9220 - accuracy: 0.0789
Epoch 23/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9217 - accuracy: 0.0800
Epoch 24/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9178 - accuracy: 0.0805
Epoch 25/100
49/49 [==============================] - 3s 53ms/step - loss: 2.9136 - accuracy: 0.0815
Epoch 26/100
49/49 [==============================] - 3s 53ms/step - loss: 2.9120 - accuracy: 0.0834
Epoch 27/100
49/49 [==============================] - 3s 53ms/step - loss: 2.9122 - accuracy: 0.0828
Epoch 28/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9087 - accuracy: 0.0844
Epoch 29/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9067 - accuracy: 0.0853
Epoch 30/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9060 - accuracy: 0.0857
Epoch 31/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9046 - accuracy: 0.0856
Epo

49/49 [==============================] - 3s 54ms/step - loss: 2.8166 - accuracy: 0.0927
Epoch 84/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8138 - accuracy: 0.0932
Epoch 85/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8149 - accuracy: 0.0938
Epoch 86/100
49/49 [==============================] - 3s 53ms/step - loss: 2.8125 - accuracy: 0.0947
Epoch 87/100
49/49 [==============================] - 3s 53ms/step - loss: 2.8096 - accuracy: 0.0952
Epoch 88/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8094 - accuracy: 0.0959
Epoch 89/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8070 - accuracy: 0.0968
Epoch 90/100
49/49 [==============================] - 3s 53ms/step - loss: 2.8034 - accuracy: 0.0971
Epoch 91/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8021 - accuracy: 0.0982
Epoch 92/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8014 - accuracy: 0.0988
Epo

Epoch 44/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8344 - accuracy: 0.0977
Epoch 45/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8308 - accuracy: 0.0993
Epoch 46/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8296 - accuracy: 0.0987
Epoch 47/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8258 - accuracy: 0.0997
Epoch 48/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8231 - accuracy: 0.1006
Epoch 49/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8226 - accuracy: 0.1003
Epoch 50/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8193 - accuracy: 0.1010
Epoch 51/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8155 - accuracy: 0.1018
Epoch 52/100
49/49 [==============================] - 3s 54ms/step - loss: 2.8145 - accuracy: 0.1018
Epoch 53/100
49/49 [==============================] - 3s 53ms/step - loss: 2.8097 - accurac

49/49 [==============================] - 3s 54ms/step - loss: 3.2692 - accuracy: 0.0543
Epoch 5/100
49/49 [==============================] - 3s 53ms/step - loss: 3.1027 - accuracy: 0.0584
Epoch 6/100
49/49 [==============================] - 3s 53ms/step - loss: 3.0556 - accuracy: 0.0609
Epoch 7/100
49/49 [==============================] - 3s 54ms/step - loss: 3.0320 - accuracy: 0.0644
Epoch 8/100
49/49 [==============================] - 3s 54ms/step - loss: 3.0160 - accuracy: 0.0656
Epoch 9/100
49/49 [==============================] - 3s 54ms/step - loss: 3.0006 - accuracy: 0.0669
Epoch 10/100
49/49 [==============================] - 3s 53ms/step - loss: 2.9827 - accuracy: 0.0700
Epoch 11/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9659 - accuracy: 0.0727
Epoch 12/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9530 - accuracy: 0.0750
Epoch 13/100
49/49 [==============================] - 3s 54ms/step - loss: 2.9433 - accuracy: 0.0744
Epoch 14

49/49 [==============================] - 3s 53ms/step - loss: 2.7992 - accuracy: 0.0988
Epoch 66/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7962 - accuracy: 0.0993
Epoch 67/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7950 - accuracy: 0.1000
Epoch 68/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7937 - accuracy: 0.1010
Epoch 69/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7922 - accuracy: 0.1019
Epoch 70/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7861 - accuracy: 0.1023
Epoch 71/100
49/49 [==============================] - 3s 53ms/step - loss: 2.7840 - accuracy: 0.1038
Epoch 72/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7820 - accuracy: 0.1041
Epoch 73/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7806 - accuracy: 0.1048
Epoch 74/100
49/49 [==============================] - 3s 54ms/step - loss: 2.7783 - accuracy: 0.1053
Epo

In [25]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(100)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

Epoch 1/100
49/49 [==============================] - 20s 275ms/step - loss: 3.6658 - accuracy: 0.0337
Epoch 2/100
49/49 [==============================] - 14s 278ms/step - loss: 3.3154 - accuracy: 0.0437
Epoch 3/100
49/49 [==============================] - 13s 270ms/step - loss: 3.0472 - accuracy: 0.0499
Epoch 4/100
49/49 [==============================] - 13s 271ms/step - loss: 3.0410 - accuracy: 0.0517
Epoch 5/100
49/49 [==============================] - 13s 262ms/step - loss: 3.0411 - accuracy: 0.0515
Epoch 6/100
49/49 [==============================] - 13s 262ms/step - loss: 3.0237 - accuracy: 0.0598
Epoch 7/100
49/49 [==============================] - 14s 278ms/step - loss: 2.8970 - accuracy: 0.0924
Epoch 8/100
49/49 [==============================] - 13s 268ms/step - loss: 2.6783 - accuracy: 0.1324
Epoch 9/100
49/49 [==============================] - 13s 267ms/step - loss: 2.6371 - accuracy: 0.1373
Epoch 10/100
49/49 [==============================] - 13s 266ms/step - loss: 2.625

49/49 [==============================] - 13s 261ms/step - loss: 2.3532 - accuracy: 0.2295
Epoch 60/100
49/49 [==============================] - 13s 261ms/step - loss: 2.3499 - accuracy: 0.2306
Epoch 61/100
49/49 [==============================] - 13s 261ms/step - loss: 2.3467 - accuracy: 0.2320
Epoch 62/100
49/49 [==============================] - 13s 261ms/step - loss: 2.3446 - accuracy: 0.2334
Epoch 63/100
49/49 [==============================] - 13s 258ms/step - loss: 2.3423 - accuracy: 0.2345
Epoch 64/100
49/49 [==============================] - 13s 259ms/step - loss: 2.3418 - accuracy: 0.2342
Epoch 65/100
49/49 [==============================] - 13s 259ms/step - loss: 2.3425 - accuracy: 0.2347
Epoch 66/100
49/49 [==============================] - 13s 258ms/step - loss: 2.3383 - accuracy: 0.2365
Epoch 67/100
49/49 [==============================] - 13s 259ms/step - loss: 2.3339 - accuracy: 0.2380
Epoch 68/100
49/49 [==============================] - 13s 262ms/step - loss: 2.3296 - 

49/49 [==============================] - 13s 266ms/step - loss: 2.5503 - accuracy: 0.1509
Epoch 18/100
49/49 [==============================] - 13s 263ms/step - loss: 2.5398 - accuracy: 0.1515
Epoch 19/100
49/49 [==============================] - 13s 262ms/step - loss: 2.5309 - accuracy: 0.1524
Epoch 20/100
49/49 [==============================] - 12s 254ms/step - loss: 2.5248 - accuracy: 0.1534
Epoch 21/100
49/49 [==============================] - 13s 258ms/step - loss: 2.5158 - accuracy: 0.1537
Epoch 22/100
49/49 [==============================] - 13s 265ms/step - loss: 2.5097 - accuracy: 0.1554
Epoch 23/100
49/49 [==============================] - 13s 263ms/step - loss: 2.5041 - accuracy: 0.1552
Epoch 24/100
49/49 [==============================] - 13s 261ms/step - loss: 2.4982 - accuracy: 0.1567
Epoch 25/100
49/49 [==============================] - 13s 260ms/step - loss: 2.4958 - accuracy: 0.1561
Epoch 26/100
49/49 [==============================] - 12s 253ms/step - loss: 2.4912 - 

49/49 [==============================] - 13s 263ms/step - loss: 2.4388 - accuracy: 0.2037
Epoch 76/100
49/49 [==============================] - 13s 264ms/step - loss: 2.4375 - accuracy: 0.2033
Epoch 77/100
49/49 [==============================] - 13s 265ms/step - loss: 2.4356 - accuracy: 0.2048
Epoch 78/100
49/49 [==============================] - 13s 264ms/step - loss: 2.4330 - accuracy: 0.2049
Epoch 79/100
49/49 [==============================] - 13s 261ms/step - loss: 2.4318 - accuracy: 0.2051
Epoch 80/100
49/49 [==============================] - 13s 261ms/step - loss: 2.4284 - accuracy: 0.2058
Epoch 81/100
49/49 [==============================] - 13s 261ms/step - loss: 2.4275 - accuracy: 0.2056
Epoch 82/100
49/49 [==============================] - 13s 261ms/step - loss: 2.4253 - accuracy: 0.2061
Epoch 83/100
49/49 [==============================] - 13s 264ms/step - loss: 2.4236 - accuracy: 0.2071
Epoch 84/100
49/49 [==============================] - 13s 267ms/step - loss: 2.4223 - 

49/49 [==============================] - 13s 257ms/step - loss: 2.4970 - accuracy: 0.1578
Epoch 34/100
49/49 [==============================] - 13s 256ms/step - loss: 2.4921 - accuracy: 0.1580
Epoch 35/100
49/49 [==============================] - 13s 257ms/step - loss: 2.4875 - accuracy: 0.1585
Epoch 36/100
49/49 [==============================] - 13s 263ms/step - loss: 2.4829 - accuracy: 0.1591
Epoch 37/100
49/49 [==============================] - 13s 257ms/step - loss: 2.4797 - accuracy: 0.1601
Epoch 38/100
49/49 [==============================] - 13s 257ms/step - loss: 2.4759 - accuracy: 0.1614
Epoch 39/100
49/49 [==============================] - 12s 253ms/step - loss: 2.4735 - accuracy: 0.1619
Epoch 40/100
49/49 [==============================] - 12s 253ms/step - loss: 2.4702 - accuracy: 0.1630
Epoch 41/100
49/49 [==============================] - 12s 249ms/step - loss: 2.4691 - accuracy: 0.1629
Epoch 42/100
49/49 [==============================] - 12s 255ms/step - loss: 2.4658 - 

49/49 [==============================] - 13s 264ms/step - loss: 2.2980 - accuracy: 0.2524
Epoch 92/100
49/49 [==============================] - 13s 266ms/step - loss: 2.2944 - accuracy: 0.2536
Epoch 93/100
49/49 [==============================] - 13s 263ms/step - loss: 2.2925 - accuracy: 0.2542
Epoch 94/100
49/49 [==============================] - 13s 263ms/step - loss: 2.2887 - accuracy: 0.2560
Epoch 95/100
49/49 [==============================] - 13s 266ms/step - loss: 2.2863 - accuracy: 0.2568
Epoch 96/100
49/49 [==============================] - 13s 266ms/step - loss: 2.2840 - accuracy: 0.2581
Epoch 97/100
49/49 [==============================] - 13s 264ms/step - loss: 2.2822 - accuracy: 0.2587
Epoch 98/100
49/49 [==============================] - 13s 269ms/step - loss: 2.2780 - accuracy: 0.2600
Epoch 99/100
49/49 [==============================] - 13s 263ms/step - loss: 2.2761 - accuracy: 0.2607
Epoch 100/100
49/49 [==============================] - 13s 272ms/step - loss: 2.2737 -

49/49 [==============================] - 13s 264ms/step - loss: 2.3783 - accuracy: 0.2151
Epoch 50/100
49/49 [==============================] - 13s 265ms/step - loss: 2.3779 - accuracy: 0.2165
Epoch 51/100
49/49 [==============================] - 13s 262ms/step - loss: 2.3749 - accuracy: 0.2176
Epoch 52/100
49/49 [==============================] - 13s 265ms/step - loss: 2.3731 - accuracy: 0.2182
Epoch 53/100
49/49 [==============================] - 13s 264ms/step - loss: 2.3716 - accuracy: 0.2192
Epoch 54/100
49/49 [==============================] - 13s 265ms/step - loss: 2.3688 - accuracy: 0.2202
Epoch 55/100
49/49 [==============================] - 13s 263ms/step - loss: 2.3676 - accuracy: 0.2213
Epoch 56/100
49/49 [==============================] - 13s 263ms/step - loss: 2.3669 - accuracy: 0.2219
Epoch 57/100
49/49 [==============================] - 13s 263ms/step - loss: 2.3690 - accuracy: 0.2212
Epoch 58/100
49/49 [==============================] - 13s 264ms/step - loss: 2.3639 - 